In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = "/content/drive/My Drive/MachineHack_1/Inputs"

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
train = pd.read_csv(os.path.join(root_path, 'Train.csv'))
test = pd.read_csv(os.path.join(root_path, 'Test.csv'))
#df_full = pd.concat([train,test], axis=0)

In [ ]:
train['prod_return'] = np.where(train.Quantity < 0, 1, 0)
test['prod_return'] = np.where(test.Quantity < 0, 1, 0)

In [ ]:
train.loc[train.Quantity < 0, 'Quantity'] = 0
test.loc[test.Quantity < 0, 'Quantity'] = 0

In [ ]:
train['UnitPrice'] = np.log1p(train['UnitPrice'])
#
train['Quantity'] = np.log1p(train['Quantity'])
test['Quantity'] = np.log1p(test['Quantity'])

In [ ]:
'''
train['customer_invoice_count'] = train.groupby(['CustomerID','InvoiceNo'])['InvoiceNo'].transform('count')
test['customer_invoice_count'] = test.groupby(['CustomerID','InvoiceNo'])['InvoiceNo'].transform('count')
#
test['customer_stock_count'] = test.groupby(['CustomerID','StockCode'])['StockCode'].transform('count')
train['customer_stock_count'] = train.groupby(['CustomerID','StockCode'])['StockCode'].transform('count')
#
train['stock_quantity_sum'] = train.groupby('StockCode')['Quantity'].transform('sum')
test['stock_quantity_sum'] = test.groupby('StockCode')['Quantity'].transform('sum')
#
train['country_customer_count'] = train.groupby(['Country','CustomerID'])['CustomerID'].transform('count')
test['country_customer_count'] = test.groupby(['Country','CustomerID'])['CustomerID'].transform('count')
#
train['country_stock_count'] = train.groupby(['Country','StockCode'])['StockCode'].transform('count')
test['country_stock_count'] = test.groupby(['Country','StockCode'])['StockCode'].transform('count')
#
train['country_quantity_sum'] = train.groupby('Country')['Quantity'].transform('sum')
test['country_quantity_sum'] = test.groupby('Country')['Quantity'].transform('sum')
#
train['invdate_quantity_sum'] = train.groupby('InvoiceDate')['Quantity'].transform('sum')
test['invdate_quantity_sum'] = test.groupby('InvoiceDate')['Quantity'].transform('sum')
#
train['Invdate'] = pd.to_datetime(train['InvoiceDate']).dt.date
test['Invdate'] = pd.to_datetime(test['InvoiceDate']).dt.date
#
train['invdate_invno_count'] = train.groupby(['Invdate','InvoiceNo'])['InvoiceNo'].transform('count')
test['invdate_invno_count'] = test.groupby(['Invdate','InvoiceNo'])['InvoiceNo'].transform('count')
#
train['invdate_stock_count'] = train.groupby(['Invdate','StockCode'])['StockCode'].transform('count')
test['invdate_stock_count'] = test.groupby(['Invdate','StockCode'])['StockCode'].transform('count')
#
train['invdate_customer_count'] = train.groupby(['Invdate','CustomerID'])['CustomerID'].transform('count')
test['invdate_customer_count'] = test.groupby(['Invdate','CustomerID'])['CustomerID'].transform('count')
#
train['Invtime'] = pd.to_datetime(train['InvoiceDate']).dt.hour
test['Invtime'] = pd.to_datetime(test['InvoiceDate']).dt.hour
#
train['invtime_invno_count'] = train.groupby(['Invtime','InvoiceNo'])['InvoiceNo'].transform('count')
test['invtime_invno_count'] = test.groupby(['Invtime','InvoiceNo'])['InvoiceNo'].transform('count')
#
train['invtime_stock_count'] = train.groupby(['Invtime','StockCode'])['StockCode'].transform('count')
test['invtime_stock_count'] = test.groupby(['Invtime','StockCode'])['StockCode'].transform('count')
#
train['invtime_customer_count'] = train.groupby(['Invtime','CustomerID'])['CustomerID'].transform('count')
test['invtime_customer_count'] = test.groupby(['Invtime','CustomerID'])['CustomerID'].transform('count')
#
train['inv_stock_count'] = train.groupby(['InvoiceNo','StockCode'])['StockCode'].transform('count')
test['inv_stock_count'] = test.groupby(['InvoiceNo','StockCode'])['StockCode'].transform('count')
'''

In [ ]:
def getDateTime(df):
  df1 = pd.to_datetime(df['InvoiceDate'])
  df['inv_year'] = df1.dt.year
  df['inv_month'] = df1.dt.month
  df['inv_day'] = df1.dt.day
  df['inv_dow'] = df1.dt.dayofweek
  df['inv_hr'] = df1.dt.hour
  df['inv_min'] = df1.dt.minute
  df['inv_monthst'] = df1.dt.is_month_start.astype('int')
  df['inv_monthed'] = df1.dt.is_month_end.astype('int')
  df['inv_yearst'] = df1.dt.is_year_start.astype('int')
  df['inv_yeared'] = df1.dt.is_year_end.astype('int')
  #df['inv_sec'] = df1.dt.second
  #return df
getDateTime(train)
getDateTime(test)

In [ ]:
df1 = train[['InvoiceNo', 'StockCode','Quantity']]
df2 = df1.append(test[['InvoiceNo', 'StockCode','Quantity']])
df3 = pd.DataFrame(df2.groupby(['InvoiceNo','StockCode'])['Quantity'].mean())
df3 = df3.reset_index()
df3 = df3.sort_values('Quantity')
df3['StockCode_group'] = df3[['InvoiceNo']].astype(str)+'_group'
df3['group'] = np.arange(len(df3))
stock_dict = dict(zip(df3.StockCode_group, df3.group))
train['StockCode_group'] = train[['InvoiceNo']].astype(str)+'_group'
test['StockCode_group'] = test[['InvoiceNo']].astype(str)+'_group'
train['StockCode_class'] = train.StockCode_group.map(stock_dict)
test['StockCode_class'] = test.StockCode_group.map(stock_dict)

In [ ]:
df1 = train[['StockCode', 'CustomerID','Quantity']]
df2 = df1.append(test[['StockCode', 'CustomerID','Quantity']])
df3 = pd.DataFrame(df2.groupby(['StockCode','CustomerID'])['Quantity'].mean())
df3 = df3.reset_index()
df3 = df3.sort_values('Quantity')
df3['Customer_group'] = df3[['StockCode']].astype(str)+'_group'
df3['group'] = np.arange(len(df3))
cust_dict = dict(zip(df3.Customer_group, df3.group))
train['Customer_group'] = train[['StockCode']].astype(str)+'_group'
test['Customer_group'] = test[['StockCode']].astype(str)+'_group'
train['Customer_class'] = train.Customer_group.map(cust_dict)
test['Customer_class'] = test.Customer_group.map(cust_dict)

In [ ]:
train['InvDate'] = pd.to_datetime(train['InvoiceDate']).dt.date
test['InvDate'] = pd.to_datetime(test['InvoiceDate']).dt.date
df1 = train[['StockCode', 'InvDate','Quantity']]
df2 = df1.append(test[['StockCode', 'InvDate','Quantity']])
df3 = pd.DataFrame(df2.groupby(['StockCode','InvDate'])['Quantity'].mean())
df3 = df3.reset_index()
df3 = df3.sort_values('Quantity')
df3['Invdate_group'] = df3[['StockCode']].astype(str)+'_group'
df3['group'] = np.arange(len(df3))
inv_dict = dict(zip(df3.Invdate_group, df3.group))
train['InvDate_group'] = train[['StockCode']].astype(str)+'_group'
test['InvDate_group'] = test[['StockCode']].astype(str)+'_group'
train['InvDate_class'] = train.InvDate_group.map(inv_dict)
test['InvDate_class'] = test.InvDate_group.map(inv_dict)

In [ ]:
'''
#after 27.07532
df1 = train[['CustomerID', 'InvoiceNo','Quantity']]
df2 = df1.append(test[['CustomerID', 'InvoiceNo','Quantity']])
df3 = pd.DataFrame(df2.groupby(['CustomerID','InvoiceNo'])['Quantity'].mean())
df3 = df3.reset_index()
df3 = df3.sort_values('Quantity')
df3['Invoice_group'] = df3[['CustomerID']].astype(str)+'_group'
df3['group'] = np.arange(len(df3))
invoice_dict = dict(zip(df3.Invoice_group, df3.group))
train['Invoice_group'] = train[['CustomerID']].astype(str)+'_group'
test['Invoice_group'] = test[['CustomerID']].astype(str)+'_group'
train['Invoice_class'] = train.Invoice_group.map(invoice_dict)
test['Invoice_class'] = test.Invoice_group.map(invoice_dict)
'''

In [ ]:
'''
#after 27.07532
train['Invhour'] = pd.to_datetime(train['InvoiceDate']).dt.hour
test['Invhour'] = pd.to_datetime(test['InvoiceDate']).dt.hour
df1 = train[['CustomerID', 'Invhour','Quantity']]
df2 = df1.append(test[['CustomerID', 'Invhour','Quantity']])
df3 = pd.DataFrame(df2.groupby(['CustomerID','Invhour'])['Quantity'].mean())
df3 = df3.reset_index()
df3 = df3.sort_values('Quantity')
df3['Invhour_group'] = df3[['CustomerID']].astype(str)+'_group'
df3['group'] = np.arange(len(df3))
invhour_dict = dict(zip(df3.Invhour_group, df3.group))
train['InvHour_group'] = train[['CustomerID']].astype(str)+'_group'
test['InvHour_group'] = test[['CustomerID']].astype(str)+'_group'
train['InvHour_class'] = train.InvHour_group.map(invhour_dict)
test['InvHour_class'] = test.InvHour_group.map(invhour_dict)
'''

In [ ]:
#pd.to_pickle(train, os.path.join(root_path, 'train_new1.pkl'))
#pd.to_pickle(test, os.path.join(root_path, 'test_new1.pkl'))

In [ ]:
#train = pd.read_pickle(os.path.join(root_path, 'train_new1.pkl'))
#test = pd.read_pickle(os.path.join(root_path, 'test_new1.pkl'))

In [ ]:
def replaceZero(df):
  stock_list = df.loc[df['UnitPrice'] == 0, 'StockCode']
  for s in stock_list:
    m1 = ((df['UnitPrice'] == 0) & (df['StockCode'] == s))
    df.loc[m1, 'UnitPrice'] = df.loc[df['StockCode'] == s, 'UnitPrice'].mean()

replaceZero(train)

In [ ]:
train['inv_year'] = train['inv_year'].map({2011:0, 2010:1})
test['inv_year'] = test['inv_year'].map({2011:0, 2010:1}) 

In [ ]:
def add_noise(series, noise_level):
  return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, test_series=None, target=None, 
                  min_samples_leaf=1, smoothing=1, noise_level=0):
  assert len(trn_series) == len(target)
  assert trn_series.name == test_series.name
  temp = pd.concat([trn_series, target], axis=1)
  averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean','count'])
  smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
  prior = target.mean()
  averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
  averages.drop(["mean", "count"], axis=1, inplace=True)
  ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name), 
                          averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
                          on=trn_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
  ft_trn_series.index = trn_series.index
  ft_test_series = pd.merge(test_series.to_frame(test_series.name), 
                            averages.reset_index().rename(columns={'index':target.name, target.name: 'average'}),
                            on=test_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
  ft_test_series.index = test_series.index
  return add_noise(ft_trn_series, noise_level), add_noise(ft_test_series, noise_level)

In [ ]:
'''
trn_invoice_class, tst_invoice_class = target_encode(train["Invoice_class"],
                         test["Invoice_class"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_invhour_class, tst_invhour_class = target_encode(train["InvHour_class"],
                         test["InvHour_class"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
                         '''

In [ ]:
trn_stockcode_class, tst_stockcode_class = target_encode(train["StockCode_class"],
                         test["StockCode_class"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_customer_class, tst_customer_class = target_encode(train["Customer_class"],
                         test["Customer_class"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_invdate_class, tst_invdate_class = target_encode(train["InvDate_class"],
                         test["InvDate_class"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_country, tst_country = target_encode(train["Country"],
                         test["Country"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_inv_month, tst_inv_month = target_encode(train["inv_month"],
                         test["inv_month"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_inv_day, tst_inv_day = target_encode(train["inv_day"],
                         test["inv_day"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_inv_dow, tst_inv_dow = target_encode(train["inv_dow"],
                         test["inv_dow"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_inv_hr, tst_inv_hr = target_encode(train["inv_hr"],
                         test["inv_hr"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)
trn_inv_min, tst_inv_min = target_encode(train["inv_min"],
                         test["inv_min"],
                         target=train.UnitPrice,
                         min_samples_leaf=100,
                         smoothing=20,
                         noise_level=0.01)

In [ ]:
train_encoded = pd.concat([trn_stockcode_class,trn_customer_class,trn_invdate_class,
                           trn_country,trn_inv_month,
                           trn_inv_day,trn_inv_dow,trn_inv_hr,trn_inv_min], axis=1)
test_encoded = pd.concat([tst_stockcode_class,tst_customer_class,tst_invdate_class,
                          tst_country,tst_inv_month,
                           tst_inv_day,tst_inv_dow,tst_inv_hr,tst_inv_min], axis=1)

In [ ]:
train.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'prod_return', 'inv_year',
       'inv_month', 'inv_day', 'inv_dow', 'inv_hr', 'inv_min', 'inv_monthst',
       'inv_monthed', 'inv_yearst', 'inv_yeared', 'StockCode_group',
       'StockCode_class', 'Customer_group', 'Customer_class', 'InvDate',
       'InvDate_group', 'InvDate_class'],
      dtype='object')

In [ ]:
col_to_drop = ['InvoiceNo', 'StockCode','InvoiceDate','CustomerID',
               'Country','StockCode_group', 'StockCode_class',
               'Customer_group','Customer_class','InvDate_group','InvDate_class',
               'InvDate','inv_month', 'inv_day', 'inv_dow', 'inv_hr', 'inv_min']
train_final = pd.concat([train, train_encoded], axis=1).reset_index(drop=True).drop(columns=col_to_drop, axis=1)
test_final = pd.concat([test, test_encoded], axis=1).reset_index(drop=True).drop(columns=col_to_drop, axis=1)

In [ ]:
train_final.columns

Index(['Description', 'Quantity', 'UnitPrice', 'prod_return', 'inv_year',
       'inv_monthst', 'inv_monthed', 'inv_yearst', 'inv_yeared',
       'StockCode_class_mean', 'Customer_class_mean', 'InvDate_class_mean',
       'Country_mean', 'inv_month_mean', 'inv_day_mean', 'inv_dow_mean',
       'inv_hr_mean', 'inv_min_mean'],
      dtype='object')

In [ ]:
test_final.columns

Index(['Description', 'Quantity', 'prod_return', 'inv_year', 'inv_monthst',
       'inv_monthed', 'inv_yearst', 'inv_yeared', 'StockCode_class_mean',
       'Customer_class_mean', 'InvDate_class_mean', 'Country_mean',
       'inv_month_mean', 'inv_day_mean', 'inv_dow_mean', 'inv_hr_mean',
       'inv_min_mean'],
      dtype='object')

In [ ]:
X = train_final.drop('UnitPrice', axis=1)
y = train_final['UnitPrice']

In [ ]:
from sklearn.preprocessing import RobustScaler
rs = RobustScaler()
transformer = rs.fit(X)
train_scaled = transformer.transform(X)
test_scaled = transformer.transform(test_final)

In [ ]:
X_final = pd.DataFrame(train_scaled, columns=test_final.columns)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)
print(f'Xtrain: {X_train.shape}')
print(f'ytrain: {y_train.shape}')
print(f'Xtest: {X_test.shape}')
print(f'ytest: {y_test.shape}')

Xtrain: (227824, 17)
ytrain: (227824,)
Xtest: (56956, 17)
ytest: (56956,)


In [ ]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
r2_train = float(format(rfr.score(X_train, y_train), '.3f'))
r2_test = float(format(rfr.score(X_test, y_test), '.3f'))
print(f'r2_train randomforest {r2_train}')
print(f'r2_test randomforest {r2_test}')

r2_train randomforest 0.99
r2_test randomforest 0.927


In [ ]:
predictions = rfr.predict(X_test)
errors = abs(predictions - y_test)
mape = 100 * np.mean(errors / y_test)
accuracy = 100 - mape
print('Model Performance')
print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Average Error: 0.0666 degrees.
Accuracy = 89.75%.


In [ ]:
from sklearn.metrics import mean_squared_error
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, predictions))}')

RMSE: 0.1651034828022791


In [ ]:
from sklearn.metrics import r2_score
print(f'r2_te: {r2_score(y_test, predictions)}')

r2_te: 0.9200344830706569


In [ ]:
X_te_final = pd.DataFrame(test_scaled, columns=test_final.columns)

In [ ]:
final_result = rfr.predict(X_te_final)
#final_df = pd.DataFrame({'UnitPrice':final_result})
sub_df = pd.DataFrame({'UnitPrice': np.expm1(final_result)})
sub_df.to_csv(os.path.join(root_path, 'submit-17.csv'), index=False)